In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd

In [3]:
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from example.config import DEFAULT_CONSTRAINTS, DEFAULT_PORTFOLIO_SETUP, DEFAULT_STRATEGY_CONFIG
from data.data import Benchmarks


scenario = Scenario(
    name="test backtest 10y",
    start_date="2015-01-01",
    end_date="2025-05-01",
    benchmark=Benchmarks.SP500,
    constraints=DEFAULT_CONSTRAINTS, 
    portfolio_config=DEFAULT_PORTFOLIO_SETUP,
    verbose=True,
    portfolio_name="sp500_10y_long_only_macd_zscore",
)  
scenario.set_strategies(DEFAULT_STRATEGY_CONFIG)
backtest = Backtest(scenario, verbose=True)


Portfolio setup: {
    "initial_capital": 100000,
    "initial_holdings": {},
    "initial_value": 0,
    "capital_growth_freq": "M",
    "new_capital_growth_pct": 0,
    "new_capital_growth_amt": 10000,
    "allocation_method": "equal",
    "excluded_sectors": [],
    "included_countries": [
        "United States"
    ],
    "min_market_cap": 0,
    "max_market_cap": Infinity,
    "trailing_stop_loss_pct": 0.05,
    "trailing_update_threshold": 0.02
}
Strategies: {
    "macd_x": true,
    "z": true
}


In [4]:
backtest.run_batch(verbose=True)

Backtest starting... swoosh!
Universe size: 480
Total trading days: 2598
Total strategies: 2
Starting in 2015-01-01
Ending in 2025-05-01


Backtesting by strategy: 100%|██████████| 2/2 [00:15<00:00,  7.57s/strategy]


Backtest completed!


In [12]:
backtest.generate_report(filename="poc_report.pdf")      

Report saved to: outputs/backtest/poc_report.pdf


'outputs/backtest/poc_report.pdf'

In [21]:
p = backtest.get_portfolio()